In [ ]:
# clean train set, clean test set

for i in range(0, 1):
    # prefix 1~30 까지의 train set을 준비
    train_df = pd.read_csv(concat_path + "\\" + 'clean' +'_ACCEPTED_concat.csv') # change (1)

    X_train = train_df.drop('label', axis=1)
    X_train = X_train.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))    
    y_train = train_df['label']

    result_df = pd.DataFrame(columns=['Train 파일 이름', 'Test 파일 이름', 'Model', 'Accuracy Score', 'AUC Score'])

    for m in range(1, 31):
        df = pd.read_csv(temp_path + '/prefix' + str(m) + '_bpic2012_O_ACCEPTED-COMPLETE_clean.csv') # change (2)

        train_file_name = 'clean' + '_ACCEPTED_concat' # change (3)
        test_file_name = "prefix" + str(m) + "_" + 'clean' + '_ACCEPTED' # change (4)

        # 각 caseid 별 첫번째 event의 timestamp에 의한 train/test set 구분
        # df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df_head = df.groupby('Case').head(1).sort_values(by='Timestamp')

        total_rows = len(df_head)
        train_length = int(total_rows * 0.8)
        train_data = df_head.iloc[:train_length]
        test_data = df_head.iloc[train_length:]

        encoder_test = Aggregate(case_id_col=['Case'], cat_cols=['Activity', 'label'], boolean=False)
        encoder_test_df = encoder_test.fit_transform(df)
        encoder_test_df['label'] = encoder_test_df['label_deviant'].apply(lambda x: 0 if x > 0 else 1)
        encoder_test_df.drop(['label_deviant', 'label_regular'], axis=1, inplace=True)
        encoder_test_df = encoder_test_df.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))    
        encoder_test_df['set'] = np.where(encoder_test_df.index.isin(train_data['Case']), 'train', 'test')

        train = encoder_test_df[encoder_test_df['set'] == 'train']
        test = encoder_test_df[encoder_test_df['set'] == 'test']

        X_test = test.drop(['label', 'set'], axis=1)
        y_test = test['label']

        # X_test의 컬럼명을 X_train의 컬럼명과 동일하게 변경, 단 X_test의 컬럼 수는 2개이고 X_train의 컬럼 수는 32개이므로 나머지 컬럼은 0으로 채움
        missing_columns = list(set(X_train.columns) - set(X_test.columns))

        # missing_columns
        for col in missing_columns:
            X_test[col] = 0

        # 컬럼을 정렬하여 순서를 맞춤
        X_test = X_test[X_train.columns]

        models = ['Random Forest', 'XGB']
        pipelines = [
                Pipeline([('model', RandomForestClassifier())]),
            Pipeline([('model', XGBClassifier())]),
            Pipeline([('model', RidgeClassifier())])
        ]
        param_grids = [
                {
                    'model__n_estimators': [5, 10, 20],
                'model__max_depth': [2, 4, 6, None],
                'model__min_samples_split': [2, 5, 10],
                'model__min_samples_leaf': [1, 2, 4],
            },
            {
                    'model__n_estimators': [5, 10, 20],
                'model__max_depth': [2, 4, 6, None],
                'model__learning_rate': [0.05, 0.1, 0.3],
                'model__min_child_weight': [1, 3, 5]
            },
        ]
        for model, pipeline, param_grid in zip(models, pipelines, param_grids):
            kf = KFold(n_splits=5, shuffle=True, random_state=2024)
            grid_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kf, scoring='accuracy')
            grid_model.fit(X_train, y_train)
            best_model = grid_model.best_estimator_

            # 테스트 세트에 모델을 적용하여 예측 수행
            y_test_pred = best_model.predict(X_test)
            accuracy = accuracy_score(y_test, y_test_pred)
            auc = roc_auc_score(y_test, y_test_pred)
            result_df = pd.concat([result_df, pd.DataFrame({'Train 파일 이름': [train_file_name], 'Test 파일 이름': [test_file_name],
                                                            'Model': [model], 'Accuracy Score': [accuracy],
                                                            'AUC Score': [auc]})], ignore_index=True)
            
    # 결과 저장
    save_path = str(result_path + "\\" + 'clean' + '_ACCEPTED.xlsx') # change (5)
    result_df.to_excel(save_path, index=False)


In [ ]:
# clean test set

for i in range(0, len(noise_type)):
    # prefix 1~30 까지의 train set을 준비
    train_df = pd.read_csv(concat_path + "\\" + noise_type[i] +'_ACCEPTED_concat.csv') # change (1)

    X_train = train_df.drop('label', axis=1)
    X_train = X_train.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))    
    y_train = train_df['label']

    result_df = pd.DataFrame(columns=['Train 파일 이름', 'Test 파일 이름', 'Model', 'Accuracy Score', 'AUC Score'])

    for m in range(1, 31):
        # 각 prefix에 대한 score 결과 구하기
        df = pd.read_csv(temp_path + '/prefix' + str(m) + '_bpic2012_O_ACCEPTED-COMPLETE_clean.csv') # change (2)

        train_file_name = noise_type[i] + '_ACCEPTED_concat' # change (3)
        test_file_name = "prefix" + str(m) + "_" + noise_type[i] + '_ACCEPTED' # change (4)

        # 각 caseid 별 첫번째 event의 timestamp에 의한 train/test set 구분
        # df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df_head = df.groupby('Case').head(1).sort_values(by='Timestamp')

        total_rows = len(df_head)
        train_length = int(total_rows * 0.8)
        train_data = df_head.iloc[:train_length]
        test_data = df_head.iloc[train_length:]

        encoder_test = Aggregate(case_id_col=['Case'], cat_cols=['Activity', 'label'], boolean=False)
        encoder_test_df = encoder_test.fit_transform(df)
        encoder_test_df['label'] = encoder_test_df['label_deviant'].apply(lambda x: 0 if x > 0 else 1)
        encoder_test_df.drop(['label_deviant', 'label_regular'], axis=1, inplace=True)
        encoder_test_df = encoder_test_df.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))    
        encoder_test_df['set'] = np.where(encoder_test_df.index.isin(train_data['Case']), 'train', 'test')

        train = encoder_test_df[encoder_test_df['set'] == 'train']
        test = encoder_test_df[encoder_test_df['set'] == 'test']

        X_test = test.drop(['label', 'set'], axis=1)
        y_test = test['label']

        # X_test의 컬럼명을 X_train의 컬럼명과 동일하게 변경, 단 X_test의 컬럼 수는 2개이고 X_train의 컬럼 수는 32개이므로 나머지 컬럼은 0으로 채움
        missing_columns = list(set(X_train.columns) - set(X_test.columns))

        # missing_columns
        for col in missing_columns:
            X_test[col] = 0

        # 컬럼을 정렬하여 순서를 맞춤
        X_test = X_test[X_train.columns]

        models = ['Random Forest', 'XGB']
        pipelines = [
                Pipeline([('model', RandomForestClassifier())]),
            Pipeline([('model', XGBClassifier())]),
            Pipeline([('model', RidgeClassifier())])
        ]
        param_grids = [
                {
                    'model__n_estimators': [5, 10, 20],
                'model__max_depth': [2, 4, 6, None],
                'model__min_samples_split': [2, 5, 10],
                'model__min_samples_leaf': [1, 2, 4],
            },
            {
                    'model__n_estimators': [5, 10, 20],
                'model__max_depth': [2, 4, 6, None],
                'model__learning_rate': [0.05, 0.1, 0.3],
                'model__min_child_weight': [1, 3, 5]
            },
        ]
        for model, pipeline, param_grid in zip(models, pipelines, param_grids):
            kf = KFold(n_splits=5, shuffle=True, random_state=2024)
            grid_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kf, scoring='accuracy')
            grid_model.fit(X_train, y_train)
            best_model = grid_model.best_estimator_
            # 테스트 세트에 모델을 적용하여 예측 수행
            y_test_pred = best_model.predict(X_test)
            accuracy = accuracy_score(y_test, y_test_pred)
            auc = roc_auc_score(y_test, y_test_pred)
            result_df = pd.concat([result_df, pd.DataFrame({'Train 파일 이름': [train_file_name], 'Test 파일 이름': [test_file_name],
                                                            'Model': [model], 'Accuracy Score': [accuracy],
                                                            'AUC Score': [auc]})], ignore_index=True)
            
    # 결과 저장
    save_path = str(result_path + "\\" + noise_type[i] + '_clean-test-set' + '_ACCEPTED.xlsx') # change (5)
    result_df.to_excel(save_path, index=False)

In [ ]:
# noise test set

for i in range(0, len(noise_type)):
    # prefix 1~30 까지의 train set을 준비
    train_df = pd.read_csv(concat_path + "\\" + noise_type[i] +'_ACCEPTED_concat.csv') # change (1)

    X_train = train_df.drop('label', axis=1)
    X_train = X_train.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))    
    y_train = train_df['label']

    result_df = pd.DataFrame(columns=['Train 파일 이름', 'Test 파일 이름', 'Model', 'Accuracy Score', 'AUC Score'])

    for m in range(1, 31):
        # 각 prefix에 대한 score 결과 구하기
        df = pd.read_csv(temp_path + '/prefix' + str(m) + '_' + noise_type[i] + '_bpic2012_O_ACCEPTED-COMPLETE_clean.csv') # change (2)

        train_file_name = noise_type[i] + '_ACCEPTED_concat' # change (3)
        test_file_name = "prefix" + str(m) + "_" + noise_type[i] + '_ACCEPTED' # change (4)

        # 각 caseid 별 첫번째 event의 timestamp에 의한 train/test set 구분
        # df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df_head = df.groupby('Case').head(1).sort_values(by='Timestamp')

        total_rows = len(df_head)
        train_length = int(total_rows * 0.8)
        train_data = df_head.iloc[:train_length]
        test_data = df_head.iloc[train_length:]

        encoder_test = Aggregate(case_id_col=['Case'], cat_cols=['Activity', 'label'], boolean=False)
        encoder_test_df = encoder_test.fit_transform(df)
        encoder_test_df['label'] = encoder_test_df['label_deviant'].apply(lambda x: 0 if x > 0 else 1)
        encoder_test_df.drop(['label_deviant', 'label_regular'], axis=1, inplace=True)
        encoder_test_df = encoder_test_df.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))    
        encoder_test_df['set'] = np.where(encoder_test_df.index.isin(train_data['Case']), 'train', 'test')

        train = encoder_test_df[encoder_test_df['set'] == 'train']
        test = encoder_test_df[encoder_test_df['set'] == 'test']

        X_test = test.drop(['label', 'set'], axis=1)
        y_test = test['label']

        # X_test의 컬럼명을 X_train의 컬럼명과 동일하게 변경, 단 X_test의 컬럼 수는 2개이고 X_train의 컬럼 수는 32개이므로 나머지 컬럼은 0으로 채움
        missing_columns = list(set(X_train.columns) - set(X_test.columns))

        # missing_columns
        for col in missing_columns:
            X_test[col] = 0

        # 컬럼을 정렬하여 순서를 맞춤
        X_test = X_test[X_train.columns]

        models = ['Random Forest', 'XGB']
        pipelines = [
                Pipeline([('model', RandomForestClassifier())]),
            Pipeline([('model', XGBClassifier())])
        ]
        param_grids = [
                {
                    'model__n_estimators': [5, 10, 20],
                'model__max_depth': [2, 4, 6, None],
                'model__min_samples_split': [2, 5, 10],
                'model__min_samples_leaf': [1, 2, 4],
            },
            {
                    'model__n_estimators': [5, 10, 20],
                'model__max_depth': [2, 4, 6, None],
                'model__learning_rate': [0.05, 0.1, 0.3],
                'model__min_child_weight': [1, 3, 5]
            },
        ]
        for model, pipeline, param_grid in zip(models, pipelines, param_grids):
            kf = KFold(n_splits=5, shuffle=True, random_state=2024)
            grid_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kf, scoring='accuracy')
            grid_model.fit(X_train, y_train)
            best_model = grid_model.best_estimator_
            # 테스트 세트에 모델을 적용하여 예측 수행
            y_test_pred = best_model.predict(X_test)
            accuracy = accuracy_score(y_test, y_test_pred)
            auc = roc_auc_score(y_test, y_test_pred)
            result_df = pd.concat([result_df, pd.DataFrame({'Train 파일 이름': [train_file_name], 'Test 파일 이름': [test_file_name],
                                                            'Model': [model], 'Accuracy Score': [accuracy],
                                                            'AUC Score': [auc]})], ignore_index=True)
            
    # 결과 저장
    save_path = str(result_path + "\\" + noise_type[i] + '_noise-test-set' + '_ACCEPTED.xlsx') # change (5)
    result_df.to_excel(save_path, index=False)

In [ ]:
# 파일 목록 생성
file_list = [
    "clean_ACCEPTED.xlsx",
    "insert0.003_clean-test-set_ACCEPTED.xlsx",
    "insert0.006_clean-test-set_ACCEPTED.xlsx",
    "insert0.009_clean-test-set_ACCEPTED.xlsx",
    "moved0.003_clean-test-set_ACCEPTED.xlsx",
    "moved0.006_clean-test-set_ACCEPTED.xlsx",
    "moved0.009_clean-test-set_ACCEPTED.xlsx",
    "replace0.003_clean-test-set_ACCEPTED.xlsx",
    "replace0.006_clean-test-set_ACCEPTED.xlsx",
    "replace0.009_clean-test-set_ACCEPTED.xlsx",
    "rework0.003_clean-test-set_ACCEPTED.xlsx",
    "rework0.006_clean-test-set_ACCEPTED.xlsx",
    "rework0.009_clean-test-set_ACCEPTED.xlsx",
    "skip0.003_clean-test-set_ACCEPTED.xlsx",
    "skip0.006_clean-test-set_ACCEPTED.xlsx",
    "skip0.009_clean-test-set_ACCEPTED.xlsx"
]

# 모든 파일을 하나의 데이터프레임으로 병합
combined_df = pd.DataFrame()

for file_name in file_list:
    file_path = os.path.join(result_path, file_name)  # 파일 경로 생성
    df = pd.read_excel(file_path)  # 파일 읽기
    combined_df = pd.concat([combined_df, df], ignore_index=True)  # 데이터프레임 병합

# 결과 출력
combined_df

In [ ]:
# 결과 저장 
combined_df.to_excel('result/merged_clean-test-set_ACCEPTED.xlsx', index=False) # change name

In [ ]:
import pandas as pd
import os

# 파일 목록 생성
file_list = [
    "clean_ACCEPTED.xlsx",
    "insert0.003_noise-test-set_ACCEPTED.xlsx",
    "insert0.006_noise-test-set_ACCEPTED.xlsx",
    "insert0.009_noise-test-set_ACCEPTED.xlsx",
    "moved0.003_noise-test-set_ACCEPTED.xlsx",
    "moved0.006_noise-test-set_ACCEPTED.xlsx",
    "moved0.009_noise-test-set_ACCEPTED.xlsx",
    "replace0.003_noise-test-set_ACCEPTED.xlsx",
    "replace0.006_noise-test-set_ACCEPTED.xlsx",
    "replace0.009_noise-test-set_ACCEPTED.xlsx",
    "rework0.003_noise-test-set_ACCEPTED.xlsx",
    "rework0.006_noise-test-set_ACCEPTED.xlsx",
    "rework0.009_noise-test-set_ACCEPTED.xlsx",
    "skip0.003_noise-test-set_ACCEPTED.xlsx",
    "skip0.006_noise-test-set_ACCEPTED.xlsx",
    "skip0.009_noise-test-set_ACCEPTED.xlsx"
]

# 모든 파일을 하나의 데이터프레임으로 병합
combined_df = pd.DataFrame()

for file_name in file_list:
    file_path = os.path.join(result_path, file_name)  # 파일 경로 생성
    df = pd.read_excel(file_path)  # 파일 읽기
    combined_df = pd.concat([combined_df, df], ignore_index=True)  # 데이터프레임 병합

# 결과 출력
combined_df

In [ ]:
# 결과 저장 
combined_df.to_excel('result/merged_noise-test-set_ACCEPTED.xlsx', index=False) # change name

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 엑셀 파일에서 데이터 불러오기

for m in range(0, 15, 3):
    for k in range(2):

        data = 'BPIC12_ACCEPTED' # change (1)
        ylabel = 'AUC Score' # 'Accuracy Score' or 'AUC Score'
         
        # a 그래프, clean test set

        result_df_a = pd.read_excel('result/merged_clean-test-set_ACCEPTED.xlsx') # change (2)
    
        # 특정 Train 파일 이름과 Model에 대한 데이터 추출
        train_file_a = ['insert0.003', 'insert0.006', 'insert0.009',
                    'moved0.003', 'moved0.006', 'moved0.009',
                    'replace0.003', 'replace0.006', 'replace0.009',
                    'rework0.003', 'rework0.006', 'rework0.009',
                    'skip0.003', 'skip0.006', 'skip0.009']
        model_a = ['Random Forest', 'XGB']
        clean_a = ['clean_ACCEPTED_concat'] # change (3)

        subset_df_a = result_df_a[(result_df_a['Train 파일 이름'].str.contains(clean_a[0])) & (result_df_a['Model'] == model_a[k])]
        accuracy_scores0_a = subset_df_a[ylabel]
        accuracy_scores0_a.index = range(1, len(accuracy_scores0_a) + 1)
    
        subset_df1_a = result_df_a[(result_df_a['Train 파일 이름'].str.contains(train_file_a[m])) & (result_df_a['Model'] == model_a[k])]
        accuracy_scores1_a = subset_df1_a[ylabel]
        accuracy_scores1_a.index = range(1, len(accuracy_scores1_a) + 1)
    
        subset_df2_a = result_df_a[(result_df_a['Train 파일 이름'].str.contains(train_file_a[m+1])) & (result_df_a['Model'] == model_a[k])]
        accuracy_scores2_a = subset_df2_a[ylabel]
        accuracy_scores2_a.index = range(1, len(accuracy_scores2_a) + 1)
    
        subset_df3_a = result_df_a[(result_df_a['Train 파일 이름'].str.contains(train_file_a[m+2])) & (result_df_a['Model'] == model_a[k])]
        accuracy_scores3_a = subset_df3_a[ylabel]
        accuracy_scores3_a.index = range(1, len(accuracy_scores3_a) + 1)
    
    
        # b 그래프, noisy test set
    
        result_df_b = pd.read_excel('result/merged_noise-test-set_ACCEPTED.xlsx') # change (4)
    
        # 특정 Train 파일 이름과 Model에 대한 데이터 추출
        train_file_b = ['insert0.003', 'insert0.006', 'insert0.009',
                    'moved0.003', 'moved0.006', 'moved0.009',
                    'replace0.003', 'replace0.006', 'replace0.009',
                    'rework0.003', 'rework0.006', 'rework0.009',
                    'skip0.003', 'skip0.006', 'skip0.009']
        model_b = ['Random Forest', 'XGB']
        clean_b = ['clean_ACCEPTED_concat'] # change (5)

        subset_df_b = result_df_b[(result_df_b['Train 파일 이름'].str.contains(clean_b[0])) & (result_df_b['Model'] == model_b[k])]
        accuracy_scores0_b = subset_df_b[ylabel]
        accuracy_scores0_b.index = range(1, len(accuracy_scores0_b) + 1)

        subset_df1_b = result_df_b[(result_df_b['Train 파일 이름'].str.contains(train_file_b[m])) & (result_df_b['Model'] == model_b[k])]
        accuracy_scores1_b = subset_df1_b[ylabel]
        accuracy_scores1_b.index = range(1, len(accuracy_scores1_b) + 1)

        subset_df2_b = result_df_b[(result_df_b['Train 파일 이름'].str.contains(train_file_b[m+1])) & (result_df_b['Model'] == model_b[k])]
        accuracy_scores2_b = subset_df2_b[ylabel]
        accuracy_scores2_b.index = range(1, len(accuracy_scores2_b) + 1)

        subset_df3_b = result_df_b[(result_df_b['Train 파일 이름'].str.contains(train_file_b[m+2])) & (result_df_b['Model'] == model_b[k])]
        accuracy_scores3_b = subset_df3_b[ylabel]
        accuracy_scores3_b.index = range(1, len(accuracy_scores3_b) + 1)


        # subplot으로 그래프 그리기
        plt.figure(figsize=(15, 5))

        # 첫 번째 그래프
        plt.subplot(1, 2, 1)
        plt.plot(accuracy_scores0_a, label=clean_a[0].split('_')[0])
        plt.plot(accuracy_scores1_a, label=train_file_a[m].split('_')[0])
        plt.plot(accuracy_scores2_a, label=train_file_a[m+1].split('_')[0])
        plt.plot(accuracy_scores3_a, label=train_file_a[m+2].split('_')[0])
        plt.xlabel('Prefix length')
        plt.ylabel(ylabel)
        plt.ylim(min(min('accuracy_scores0_a'), min('accuracy_scores0_b')), max(max('accuracy_scores0_a'), max('accuracy_scores0_b')))
        plt.yscale('linear')
        plt.title('Clean Test Set ' + ylabel + f' for Model: {model_a[k]}')
        plt.legend()
        plt.grid(True)

        # 두 번째 그래프
        plt.subplot(1, 2, 2)
        plt.plot(accuracy_scores0_b, label=clean_b[0].split('_')[0])
        plt.plot(accuracy_scores1_b, label=train_file_b[m].split('_')[0])
        plt.plot(accuracy_scores2_b, label=train_file_b[m+1].split('_')[0])
        plt.plot(accuracy_scores3_b, label=train_file_b[m+2].split('_')[0])
        plt.xlabel('Prefix length')
        plt.ylabel(ylabel)
        plt.yscale('linear')
        plt.ylim(min(min('accuracy_scores0_a'), min('accuracy_scores0_b')), max(max('accuracy_scores0_a'), max('accuracy_scores0_b')))
        plt.title('Noise Test Set ' + ylabel + f' for Model: {model_b[k]}')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        # plt.savefig('graph/' + data + '_' + ylabel + '_' + model_a[k] +'_' + train_file_b[m].split('_')[0] + '.png')
        plt.show()